In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import utils
import plot_utils

# Read data

In [3]:
data_file = './data/paleo_meso_ceno_data_paleogeography_epoch.csv'

data = pd.read_csv(data_file)
data = data.sort_values(by=['mean_ma'], ascending=False)

time_file = './data/timeScale.xlsx'
time = pd.read_excel(time_file,sheet_name='Sheet1')
time_period = utils.get_timesubset(time,3,541)
width_period = [time_period['max_ma'][i] - time_period['min_ma'][i] for i in list(time_period.index)]

time_epoch = utils.get_timesubset(time,4,541)
width_epoch = [time_epoch['max_ma'][i] - time_epoch['min_ma'][i] for i in list(time_epoch.index)]


# Load Sepkoski data

In [4]:
sepkoski_diversity = utils.load_sepkoski_diversity_data()

# Skeletal abundance

In [5]:
data_skeletal = utils.get_stats(data,'skeletal_total','epoch')
data_skeletal = data_skeletal.sort_values(by=['time'], ascending=True)

In [6]:
data_animal = utils.get_stats(data,'total_animals_new','epoch')

data_animal = data_animal.sort_values(by=['time'], ascending=True)

# All litho diversity

In [7]:
animal_diversity = pd.read_csv('./data/SQS_diversity_data/animal_diversity.csv')

# Plot

In [8]:
fig = make_subplots(rows = 3,cols = 1,shared_xaxes = True,row_heights = [0.45,0.45,0.1],
                    specs=[[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": False}]],
                 vertical_spacing = 0.005)
animal_color = '#ff0000'
data_color = '#5299FF'

fig.add_trace(go.Scatter(x = data_skeletal['time'],y = data_skeletal['mean'],mode = 'lines+markers',
                         name = 'All data mean',showlegend=False,line=dict(dash='dash'),
                         marker = dict(color = 'black',size = 8,),
                         ),row=1,col=1, secondary_y=False)

fig.add_trace(go.Scatter(x = sepkoski_diversity['mean_ma'],y = sepkoski_diversity['diversity'],mode = 'lines',
                         name = 'Sepkoski\'s compendium', showlegend=True,legendgroup='1',
                         line = dict(color = data_color, width=3),
                         ),row=1,col=1, secondary_y = True)


fig.add_trace(go.Scatter(x=data_animal['time'],y=data_animal['mean'],
                         mode='lines+markers',showlegend=False,marker_color='black',line=dict(dash='dash'),
                         #line=dict(thickness=)
                        ),row=2,col=1)

fig.add_trace(go.Scatter(x=animal_diversity.mean_ma,y=animal_diversity['SQS_diversity'],
                         mode='lines',name = 'PBDB dataset', showlegend=True,marker_color=animal_color,
                         line_width=3, legendgroup='2',
                        ),secondary_y=True,row=2,col=1)


plot_utils.plot_time(fig,time_period,width_period,3,1)


fig.update_xaxes(ticks = 'outside',range=(541,-2),showticklabels = False,showline = True, 
                     mirror = True,linecolor = 'black',row=1,col=1)
fig.update_xaxes(ticks = 'outside',range=(541,-2),showticklabels = False,showline = True, 
                     mirror = True,linecolor = 'black',row=2,col=1)

fig.update_xaxes(range=(541,-2),ticks = 'outside',title = 'Time (Ma)',row=3,col=1)


fig.update_yaxes(ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=1,col=1)
fig.update_yaxes(ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=2,col=1)

fig.update_layout(height = 850, width = 950,font_color = 'black', font_family='Helvetica',font_size=14,
                 paper_bgcolor = 'white',
                 plot_bgcolor = 'white',
                  yaxis1=dict(title='Skeletal biomass %',range=(0,85)),
                  yaxis2=dict(title='Animal Diversity [genera]',title_font_family='Helvetica',
                             titlefont=dict(color=data_color),tickfont=dict(color=data_color)),
                  yaxis3=dict(range=(0,52),title='Animal biomass %'),
                  yaxis4=dict(range=(0,710),title='Animal Diversity [genera]',
                              titlefont=dict(color=animal_color),tickfont=dict(color=animal_color)),
                legend_tracegroupgap = 285,

                 legend=dict(bgcolor='rgba(0,0,0,0)',
                yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=0.01))
fig.show()
fig.write_image('./figures/Figure_4.png',scale = 6)